# Assignment 2: Exploring fairness during model training

We discussed a number of fairness metrics in the context of binary classification. Building up on the previous assignment, we will explore aspects of fairness in model predictions while training our model.

In this lab, we will detect bias that may be introduced while training classifiers.

This notebook has four stages in which we will: 
1. Import the data
2. Implement a few pre-processing steps, and inspect the data (compute base rates on original data)
3. Train a classifier to predict credit using the original data, and measure bias using fairness metrics including statistical parity and equalized odds. 
4. Train a classifier to predict credit using the original data without sensitive features, and measure bias using fairness metrics including statistical parity and equalized odds. 
5. (Extra credit) Use IBM's AIF360 toolkit to compute fairness metrics.

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline

import random
random.seed(6)

import sys
import warnings

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

### Step 1: Load data

For this assignment, we will work with the German Credit dataset which has been provided with this notebook (the dataset can also be downloaded from the UCI ML repository). The dataset has demographic and financial information for about 1,000 individuals, and the task for this dataset is to predict whether an individual is a good credit risk or a bad credit risk. More information about the dataset can be found here: https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data.

Load the dataset and check the first few rows:

In [ ]:
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment',\
            'install_rate', 'personal_status', 'debtors', 'residence', 'property', 'age', 'install_plans',\
            'housing', 'num_credits', 'job', 'num_liable', 'telephone', 'foreign_worker', 'credit']
data_df = pd.read_table('german.data', names=cols, sep=" ", index_col=False)
y = data_df['credit']

print("Shape: ", data_df.shape)
data_df.head(5)

### Step 2: Data preprocessing and data analysis

#### 2.1. Adding two new columns
Before training a model, repeat the data processing steps from assignment 1 here to create the "sex" and "age" columns.
<ol>
    <li> Write code to create the "sex" column and append to the dataframe. Consider sex=`male' as privileged. </li> 
    <li> Write code to replace the "age" column with discrete values. Consider age ≥ 25 as privileged. </li>
</ol>

As in assignment 1, we will train a machine learning model to predict good/bad credit. You are free to use any model but you may use the data preprocessing steps from assignment 1 to transform the data.

In [ ]:
# Write code for data preparation here

#### 2.2. Sensitive attributes

In the class, we learnt about sensitive attributes for computing model fairness. In this assignment, we will compute different fairness metrics in two scenarios:

<ol>
    <li> considering only "age" as the sensitive attribute </li>
    <li> considering only "sex" as the sensitive attribute </li>
</ol>

#### 2.3. Compute base rates on original data
Before we consider model fairness, let's first compare the base rates for the privileged and unprivileged groups with that of the entire data. Base rate indicates the fraction of data (or a subset of the data) that has positive outcomes. Base rates indicate class imbalance in the dataset.

In the following code block, compute the base rates for the entire dataset, and for the privileged and unprivileged groups.

In [ ]:
# Write code to compute base rates with respect to "age" as the sensitive attribute

**TODO:** Write in words if the value computed above indicates favoring the privileged group or the unprivileged group.

In [ ]:
# Write code to compute base rates with respect to "sex" as the sensitive attribute

**TODO:** Write in words if the value computed above indicates favoring the privileged group or the unprivileged group.

### Step 3: Model training
Let's now prepare the setup for model training and evaluation

#### 3.1. Identify the training features and target variable

We will evaluate the learned model on the training data. Therefore, we will not split the data into train and test sets. We are also not concerned with optimizing the learned model for performance and, therefore, we will not need to create a validation dataset.

In [ ]:
# Set up the data for training
x_train = data_df.drop("credit", axis=1)
y_train = data_df.credit.replace({2:0}) #1 = Good, 2= Bad credit risk
print("Training Outcomes: \n", y_train.value_counts())

#### 3.2. Model training

In this part, we will set up our machine learning model and fit the model using the training data. Below, we have learnt a logistic regression model, but **you are free to choose any model**. Since we are not concerned with optimal model performance here, there is no need to perform hyperparameter tuning.

In [ ]:
model = LogisticRegression(C=0.5, penalty="l1", solver='liblinear')
    
# Fit the model using the training data
model = model.fit(x_train, y_train, sample_weight=None)

# Observe the class labels
print(model.classes_)

# Calculate predicted values on training data
y_pred = model.predict(x_train)

# You can also compute the probabilties for the two clases using the predict_proba function
y_pred_probs = model.predict_proba(x_train)

#### 3.3. Model evaluation

Compute the model accuracy using sklearn's `accuracy_score` function

In [ ]:
# Write code to compute model accuracy

#### 3.4. Evaluate model fairness 
In the class, we went over a number of fairness metrics. In this part of the assignment, you will write functions to compute statistical parity and equalized odds. Recall that both of these metrics use the model predictions $\hat{Y}$ (computed as $y\_pred$ above).

Let $S=0$ indicate the unprivileged group and $S=1$ indicate the privileged group. FPR and FNR represent the false positive rate and false negative rate respectively. Then, we can compute:

**Statistical parity** = $P(\hat{Y} = 1 | S = 0) - P(\hat{Y} = 1 | S = 1)$ 

**Equalized odds** $\dfrac{(FPR_{S=0} - FPR_{S=1}) + (FNR_{S=0} - FNR_{S=1})}{2}$

where $FPR = P(\hat{Y}=1 | Y=0)$ and $FNR = P(\hat{Y}=0 | Y=1)$



In [ ]:
# Write function to compute statistical parity over model predictions with given sensitive attribute

# Write function to compute equalized odds with given sensitive attribute

# Report statistical parity and equalized odds using "sex" as the sensitive attribute

# Report statistical parity and equalized odds using "age" as the sensitive attribute

**TODO:** What can you say about the model being more or less biased than the original credit scores?


### Step 4: Model training without using the sensitive feature

We discussed in the class how removing a sensitive attribute is not enough to generate models that result in fair predictions. In this step, we will see if that is indeed true in action.

4.1. In the following, we will build models trained without the sensitive attribute (once for the "sex" attribute and one for the "age" attribute). 

In [ ]:
# Drop the sensitive attribute from training data
x_train_no_age = x_train.drop('age', axis=1)
x_train_no_sex = x_train.drop('sex', axis=1)

model = LogisticRegression(C=0.5, penalty="l1", solver='liblinear')
    
# Fit the model using the training data
model_no_age = model.fit(x_train_no_age, y_train, sample_weight=None)
model_no_sex = model.fit(x_train_no_sex, y_train, sample_weight=None)

# Observe the class labels
print(model.classes_)

# Calculate the predicted values and probabilities on the updated training data
y_pred_no_age = 
y_pred_no_sex = 

#### 4.2. Evaluating updated model

Now, let's evaluate the predictions obtained from the updated model using the functions that your wrote in Step 3.4. Although the sensitive attribute has not been used in training the model, the group memberships will be used to compute the fairness metrics.

In [ ]:
# Report statistical parity and equalized odds using "sex" as the sensitive attribute

# Report statistical parity and equalized odds using "age" as the sensitive attribute

**TODO:** Compare the metrics obtained here with those obtained in Step 3.4. What can you say about whether or not using the sensitive attribute have an effect on bias in model predictions for this example?


### (**Extra Credit**) Step 5: Using AIF360 to compute fairness metrics

In this step, we will use the "aif360" package to detect bias in the dataset (AIF360 is IBM's AI Fairness toolkit) to evaluate fairness of the learned model. This package requires the data to be in a certain format, which will be clear as we walk through the following.

In [ ]:
# You should need to run this package installation only once. After the first time, you can comment it out
# !pip install aif360==0.2.2

In [ ]:
from aif360.datasets import GermanDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing

from aif360.explainers import MetricTextExplainer, MetricJSONExplainer

 Let us use "age" as the sensitive attribute and defined privileged and unprivileged values. We use the GermanDataset in aif360 to do our analysis.

In [ ]:
# Store definitions of privileged and unprivileged groups
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_orig = GermanDataset(protected_attribute_names=['age'],           
                             privileged_classes=[lambda x: x >= 25], 
                             features_to_drop=['personal_status', 'sex'])      # age >=25 is considered privileged


We will convert the dataset to a Pandas dataframe, extract the input features (X) and target variable (y).

In [ ]:
dataset_orig_df = dataset_orig.convert_to_dataframe()[0]
x_train = dataset_orig_df.drop("credit", axis=1)
y_train = dataset_orig_df.credit.replace({2:0}) 

First, we will use our model to calculate predicted values for the data and attach them as a new column in the data frame. 

In [ ]:
# Copy the dataset
preds_df = dataset_orig_df.copy()

model = LogisticRegression(C=0.5, penalty="l1", solver='liblinear')
model.fit(x_train, y_train)

# Calculate predicted values
preds_df['credit'] = model.predict(x_train)

# Recode the predictions so that they match the format that the dataset was originally provided in 
# (1 = good credit, 2 = bad credit)
preds_df['credit'] = preds_df.credit.replace({0:2})

Then we will create an object of the aif360 StandardDataset class. You can read more about this in the documentation:
https://aif360.readthedocs.io/en/latest/modules/standard_datasets.html#aif360.datasets.StandardDataset

In [ ]:
orig_aif360 = StandardDataset(dataset_orig_df, label_name='credit', protected_attribute_names=['age'], 
                privileged_classes=[[1]], favorable_classes=[1])

preds_aif360 = StandardDataset(preds_df, label_name='credit', protected_attribute_names=['age'], 
                privileged_classes=[[1]], favorable_classes=[1])

Let us calculate some fairness metrics on the training data using the `ClassificationMetric` function in aif360. 

In [ ]:
orig_vs_preds_metrics = ClassificationMetric(orig_aif360, preds_aif360,
                                                   unprivileged_groups=unprivileged_groups,
                                                   privileged_groups=privileged_groups)

In [ ]:
# For example, we can compute statistical parity difference as below:
print("Statistical parity difference: ", orig_vs_preds_metrics.statistical_parity_difference())

**TODO:** Compare the value obtained above with those obtained in steps 4.2 and 3.4 above. You might observe a difference in the performance due to the way the dataset has been created within aif360.

Feel free to explore some of the other fairness metrics that we discussed in class. You can find a list of fairness metrics currently supported by aif360 here: https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html.

# Submitting this Assignment Notebook

Once complete, please submit your assignment notebook as an attachment under "Assignments > Assignment 2" on Brightspace. You can download a copy of your notebook using ```File > Download .ipynb```. Please ensure you submit the `.ipynb` file (and not a `.py` file).